
# 📈 Weekly Performance Report

This notebook generates a **weekly performance pack** with:
- Weekly & YTD returns
- Cumulative equity curve
- PnL attribution by **strategy / sector / asset**
- Rolling Sharpe (12w), drawdowns
- Top/Worst contributors
- Exportable summary tables

> It auto-detects data in `data/` and falls back to **synthetic** data if files are missing.


## 0) Parameters

In [ ]:

# Input files (CSV). Wide formats are supported where appropriate.
PATH_RETURNS   = "data/returns.csv"      # wide: date, TICK1, TICK2, ... (daily returns in decimal)
PATH_PRICES    = "data/prices.csv"       # wide: date, TICK1, TICK2, ... (daily prices) as fallback
PATH_POSITIONS = "data/positions.csv"    # columns: ticker, quantity, avg_price, sector, strategy (optional)
PATH_MAP       = "data/asset_map.csv"    # optional: ticker -> sector,strategy mapping

# Report settings
ROLLING_WEEKS_SHARPE = 12
TOP_K = 10     # top contributors to show
WEEK_FREQ = "W-FRI"   # weekly close day

# Output
OUT_DIR = "reports"
SUMMARY_CSV = "weekly_summary.csv"


## 1) Setup & Helpers

In [ ]:

import os, math
from typing import List, Dict, Any, Optional
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.float_format = "{:,.4f}".format

def load_wide_csv(path: str) -> Optional[pd.DataFrame]:
    if os.path.exists(path):
        df = pd.read_csv(path, parse_dates=["date"]).set_index("date").sort_index()
        return df
    return None

def to_weekly(df: pd.DataFrame, method: str = "sum", freq: str = "W-FRI") -> pd.DataFrame:
    # Returns: sum (approx for small daily returns), Prices: last
    if method == "sum":
        return df.resample(freq).sum(min_count=1)
    elif method == "last":
        return df.resample(freq).last()
    else:
        raise ValueError("Unknown method")

def drawdown_curve(nav: pd.Series) -> pd.Series:
    rm = nav.cummax()
    return nav / rm - 1.0

def ann_sharpe(ret: pd.Series, periods_per_year=52) -> float:
    mu = ret.mean() * periods_per_year
    sd = ret.std(ddof=1) * math.sqrt(periods_per_year)
    return float(mu / sd) if sd > 0 else float("nan")

def rolling_sharpe(ret_w: pd.Series, window: int, periods_per_year=52) -> pd.Series:
    mu = ret_w.rolling(window).mean() * periods_per_year
    sd = ret_w.rolling(window).std(ddof=1) * math.sqrt(periods_per_year)
    rs = mu / sd
    return rs


## 2) Load Data (with synthetic fallback)

In [ ]:

# Try returns first (preferred), fallback to prices -> pct_change, else synthesize
rets = load_wide_csv(PATH_RETURNS)
if rets is None:
    prices = load_wide_csv(PATH_PRICES)
    if prices is not None:
        rets = prices.pct_change().dropna(how="all").fillna(0.0)

if rets is None or rets.empty:
    # Synthetic: 16 assets, sector/strategy groups
    rng = np.random.default_rng(7)
    dates = pd.bdate_range("2024-01-01", periods=380)
    n = 16
    # Sector drifts to create structure
    sectors = np.array(["EQT","EQT","EQT","EQT","BOND","BOND","CMD","CMD","FX","FX","CRY","CRY","ALT","ALT","VOL","VOL"])
    base_mu = {"EQT":0.0003,"BOND":0.0001,"CMD":0.0002,"FX":0.0000,"CRY":0.0006,"ALT":0.0002,"VOL":0.0002}
    base_sd = {"EQT":0.012,"BOND":0.004,"CMD":0.009,"FX":0.006,"CRY":0.03,"ALT":0.01,"VOL":0.015}
    data = []
    for i in range(n):
        sec = sectors[i]
        mu = base_mu[sec]; sd = base_sd[sec]
        data.append(rng.normal(mu, sd, size=len(dates)))
    cols = [f"T{i:02d}" for i in range(n)]
    rets = pd.DataFrame(np.array(data).T, index=dates, columns=cols)

# Positions / mapping
pos = load_wide_csv(PATH_POSITIONS)
if pos is not None:
    # if loaded as wide by mistake, fix
    if "ticker" not in pos.columns:
        pos = None

mapping = load_wide_csv(PATH_MAP)
if mapping is not None and "ticker" not in mapping.columns:
    mapping = None

# Build a join map: ticker -> sector,strategy
map_df = None
if mapping is not None:
    map_df = mapping[["ticker"] + [c for c in ["sector","strategy"] if c in mapping.columns]].set_index("ticker")

elif pos is not None:
    map_df = pos.set_index("ticker")[[c for c in ["sector","strategy"] if c in pos.columns]]

else:
    # Synthetic map
    tickers = rets.columns.tolist()
    sectors = ["EQT","BOND","CMD","FX","CRY","ALT","VOL"]
    strategies = ["StatArb","MacroRV","Event","Carry","Momentum"]
    rng = np.random.default_rng(11)
    sec = rng.choice(sectors, size=len(tickers))
    strat = rng.choice(strategies, size=len(tickers))
    map_df = pd.DataFrame({"sector": sec, "strategy": strat}, index=tickers)

rets = rets.loc[:, rets.columns.intersection(map_df.index)].copy() # type: ignore
map_df = map_df.loc[rets.columns]
rets.head()


## 3) Weekly Portfolio Returns & Equity

In [ ]:

# Equal-weight portfolio unless weights provided
weights = pd.Series(1.0 / rets.shape[1], index=rets.columns) # type: ignore

rets_w = to_weekly(rets, method="sum", freq=WEEK_FREQ).fillna(0.0) # type: ignore
port_w = (rets_w * weights).sum(axis=1)

equity_w = (1 + port_w).cumprod()
dd_w = drawdown_curve(equity_w)

summary = {
    "Weeks": int(port_w.shape[0]),
    "YTD Return": float((1 + port_w[port_w.index.year == port_w.index[-1].year]).prod() - 1.0), # type: ignore
    "Ann. Sharpe (weekly)": ann_sharpe(port_w),
    "Max Drawdown": float(dd_w.min())
}
summary


## 4) Plots

In [ ]:

plt.figure(figsize=(10,3.5))
port_w.plot(kind="bar")
plt.title("Weekly Returns")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,3.5))
equity_w.plot()
plt.title("Cumulative Equity (Weekly)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,3.5))
rolling_sharpe(port_w, ROLLING_WEEKS_SHARPE).plot()
plt.title(f"Rolling Sharpe ({ROLLING_WEEKS_SHARPE}w)")
plt.tight_layout()
plt.show()


## 5) PnL Attribution

In [ ]:

# Approx weekly contribution = weight * weekly return (since equal weights)
contrib_w = rets_w.mul(weights, axis=1)

# By strategy
if "strategy" in map_df.columns: # type: ignore
    strat_map = map_df["strategy"] # type: ignore
    by_strat = contrib_w.groupby(by=strat_map, axis=1).sum() # type: ignore
else:
    by_strat = pd.DataFrame()

# By sector
if "sector" in map_df.columns: # type: ignore
    sect_map = map_df["sector"] # type: ignore
    by_sect = contrib_w.groupby(by=sect_map, axis=1).sum() # type: ignore
else:
    by_sect = pd.DataFrame()

# Totals (YTD)
ytd_mask = by_strat.index.year == by_strat.index[-1].year if not by_strat.empty else None # type: ignore
strat_ytd = by_strat[ytd_mask].sum().sort_values(ascending=False) if not by_strat.empty else pd.Series(dtype=float)
sect_ytd  = by_sect[ytd_mask].sum().sort_values(ascending=False) if not by_sect.empty else pd.Series(dtype=float)

display(strat_ytd.to_frame("YTD Contribution"))
display(sect_ytd.to_frame("YTD Contribution"))

# Top/Worst assets YTD
asset_ytd = contrib_w[ytd_mask].sum().sort_values(ascending=False)
top_assets = asset_ytd.head(TOP_K)
worst_assets = asset_ytd.tail(TOP_K)

display(top_assets.to_frame("Top Assets YTD"))
display(worst_assets.to_frame("Worst Assets YTD"))


### Stacked Weekly Contribution — Strategy

In [ ]:

if not by_strat.empty:
    plt.figure(figsize=(10,3.8))
    by_strat.plot(kind="bar", stacked=True, ax=plt.gca())
    plt.title("Weekly Contribution by Strategy (stacked)")
    plt.tight_layout()
    plt.show()


### Stacked Weekly Contribution — Sector

In [ ]:

if not by_sect.empty:
    plt.figure(figsize=(10,3.8))
    by_sect.plot(kind="bar", stacked=True, ax=plt.gca())
    plt.title("Weekly Contribution by Sector (stacked)")
    plt.tight_layout()
    plt.show()


## 6) Weekly Summary Table

In [ ]:

weekly_tbl = pd.DataFrame({
    "weekly_return": port_w,
    "equity": equity_w,
    "drawdown": dd_w,
    "rolling_sharpe": rolling_sharpe(port_w, ROLLING_WEEKS_SHARPE)
})

os.makedirs(OUT_DIR, exist_ok=True)
out_csv = os.path.join(OUT_DIR, SUMMARY_CSV)
weekly_tbl.to_csv(out_csv, index=True)
print("Wrote summary:", out_csv)

weekly_tbl.tail(10)


## 7) Snapshot Metrics

In [ ]:

snap = {
    "as_of": str(weekly_tbl.index[-1].date()) if not weekly_tbl.empty else None,
    "weeks_in_sample": int(len(weekly_tbl)),
    "ytd_return": summary["YTD Return"],
    "ann_sharpe_weekly": summary["Ann. Sharpe (weekly)"],
    "max_drawdown": summary["Max Drawdown"],
    "top_assets_ytd": top_assets.to_dict() if 'top_assets' in locals() else {},
    "worst_assets_ytd": worst_assets.to_dict() if 'worst_assets' in locals() else {},
    "by_strategy_ytd": strat_ytd.to_dict() if 'strat_ytd' in locals() else {},
    "by_sector_ytd": sect_ytd.to_dict() if 'sect_ytd' in locals() else {},
}
snap
